In [ ]:
# Example 32-bit input
plaintext = 0x1269978  # Example input
print(f"Original Data: {hex(plaintext)}")

# Complete DES Algorithm
#des_sbox = [i ^ 0xBA for i in range(256)]  # Can be changed
inverse_des_sbox = {v: k for k, v in enumerate(des_sbox)}

def des_sbox_substitution(value):
    byte1 = (value >> 8) & 0xFF
    byte2 = value & 0xFF
    return (des_sbox[byte1] << 8) | des_sbox[byte2]

def inverse_des_sbox_substitution(value):
    byte1 = (value >> 8) & 0xFF
    byte2 = value & 0xFF
    return (inverse_des_sbox[byte1] << 8) | inverse_des_sbox[byte2]

def des_permutation(value):
    permuted = 0
    permutation_table = [15, 6, 11, 3, 8, 0, 12, 4, 7, 14, 1, 10, 2, 9, 13, 5]
    for i, bit_pos in enumerate(permutation_table):
        if value & (1 << bit_pos):
            permuted |= (1 << i)
    return permuted

def inverse_des_permutation(value):
    inverse_permuted = 0
    permutation_table = [15, 6, 11, 3, 8, 0, 12, 4, 7, 14, 1, 10, 2, 9, 13, 5]
    for i, bit_pos in enumerate(permutation_table):
        if value & (1 << i):
            inverse_permuted |= (1 << bit_pos)
    return inverse_permuted

def des_key_mixing(value, round_key):
    return value ^ round_key

# Complete AES Algorithm
inv_aes_sbox = {v: k for k, v in enumerate(aes_sbox)}

def aes_substitute(byte):
    return aes_sbox[byte]

def inverse_aes_substitute(byte):
    return inv_aes_sbox[byte]

def aes_shift_rows(value):
    byte1 = (value >> 8) & 0xFF
    byte2 = value & 0xFF
    return (byte2 << 8) | byte1

def inverse_aes_shift_rows(value):
    byte1 = (value >> 8) & 0xFF
    byte2 = value & 0xFF
    return (byte1 << 8) | byte2

def gf_mult(a, b):
    p = 0
    for _ in range(8):
        if b & 1:
            p ^= a
        hi_bit_set = a & 0x80
        a <<= 1
        if hi_bit_set:
            a ^= 0x1B
        b >>= 1
    return p & 0xFF

def aes_mix_columns(value):
    byte1 = (value >> 8) & 0xFF
    byte2 = value & 0xFF
    mixed1 = gf_mult(byte1, 2) ^ gf_mult(byte2, 3)
    mixed2 = gf_mult(byte2, 2) ^ gf_mult(byte1, 3)
    return (mixed1 << 8) | mixed2

def inverse_aes_mix_columns(value):
    byte1 = (value >> 8) & 0xFF
    byte2 = value & 0xFF
    inv_mixed1 = (gf_mult(byte1, 0x0E) ^ gf_mult(byte2, 0x0B) ^
                  gf_mult(byte1, 0x0D) ^ gf_mult(byte2, 0x09))
    inv_mixed2 = (gf_mult(byte2, 0x0E) ^ gf_mult(byte1, 0x0B) ^
                  gf_mult(byte2, 0x0D) ^ gf_mult(byte1, 0x09))
    return ((inv_mixed1 & 0xFF) << 8) | (inv_mixed2 & 0xFF)

# Generate 12 round keys from a base key
def generate_round_keys(base_key, num_rounds=12):
    return [(base_key ^ (i * 0x9F37)) & 0xFFFF for i in range(num_rounds)]

# AES-DES Encryption
def aes_des_encrypt(data, round_keys, num_rounds=12):
    left, right = (data >> 16) & 0xFFFF, data & 0xFFFF
    print(f"Initial Data: {hex(data)}")

    for round_num in range(num_rounds):
        round_key = round_keys[round_num]

        left = aes_substitute(left & 0xFF) | (aes_substitute((left >> 8) & 0xFF) << 8)
        left = aes_shift_rows(left)
        left = aes_mix_columns(left)
        left ^= round_key

        right = des_sbox_substitution(right)
        right = des_permutation(right)
        right = des_key_mixing(right, round_key)

        left, right = right, left

        print(f"After Round {round_num + 1}: {hex((left << 16) | right)}")

    encrypted = (left << 16) | right
    print(f"Final Encrypted Data: {hex(encrypted)}\n")
    return encrypted

# AES-DES Decryption
def aes_des_decrypt(data, round_keys, num_rounds=12):
    left, right = (data >> 16) & 0xFFFF, data & 0xFFFF
    print(f"Encrypted Data: {hex(data)}")

    for round_num in reversed(range(num_rounds)):
        round_key = round_keys[round_num]

        left, right = right, left
        right = des_key_mixing(right, round_key)
        right = inverse_des_permutation(right)
        right = inverse_des_sbox_substitution(right)

        left ^= round_key
        left = inverse_aes_mix_columns(left)
        left = inverse_aes_shift_rows(left)
        left = inverse_aes_substitute(left & 0xFF) | (inverse_aes_substitute((left >> 8) & 0xFF) << 8)

        print(f"After Round {num_rounds - round_num}: {hex((left << 16) | right)}")

    decrypted = (left << 16) | right
    print(f"Final Decrypted Data: {hex(decrypted)}\n")
    return decrypted

# RSA Key Generation and Operations
def generate_rsa_keys():
    p, q = 104729, 99991
    n = p * q
    phi = (p - 1) * (q - 1)
    e = 65537
    d = mod_inverse(e, phi)
    return (n, e, d)

def mod_inverse(a, m):
    m0, x0, x1 = m, 0, 1
    while a > 1:
        q = a // m
        m, a = a % m, m
        x0, x1 = x1 - q * x0, x0
    return x1 + m0 if x1 < 0 else x1

def rsa_encrypt(data, e, n):
    return pow(data, e, n)

def rsa_decrypt(data, d, n):
    return pow(data, d, n)



# Generate RSA keys
n, e, d = generate_rsa_keys()

# Generate base encryption key and round keys
base_key = 0xA3F4B67C
round_keys = generate_round_keys(base_key)

print(f"Base Encryption Key: {hex(base_key)}")
print(f"Round Keys: {[hex(k) for k in round_keys]}\n")

# Encrypt and Decrypt
encrypted_data = aes_des_encrypt(plaintext, round_keys)
rsa_encrypted = rsa_encrypt(encrypted_data, e, n)
print(f"RSA Encrypted Data: {hex(rsa_encrypted)}\n")

rsa_decrypted = rsa_decrypt(rsa_encrypted, d, n)
print(f"RSA Decrypted Data (before AES-DES decryption): {hex(rsa_decrypted)}\n")

decrypted_data = aes_des_decrypt(rsa_decrypted, round_keys)

assert decrypted_data == plaintext, "Decryption failed!"
print("Decryption successful! ✅")
